In [1]:
ipython = get_ipython()
colab   = 'google.colab' in str(ipython)
print(ipython)

if colab:
  from google.colab import drive
  drive.mount('/content/gdrive')
  %cd '/content/gdrive/My Drive/Colab Notebooks'

!ls "./"

def MODULE_FROM_NOTEBOOK(target, source, *keys): 
    from json import load
    with open(source) as notebook:
        data = load(notebook)
        with open(target,'w') as module:
          for cell in data['cells']:
              lines = cell['source']
              if type(lines) == str:
                  lines = lines.split('\n')
              line0  = (lines or [''])[0].strip()
              haskey = lambda key: line0 == f'### {key} ###'
              if cell['cell_type'] == 'code' and any(map(haskey,keys)):
                  code = '\n'.join(lines)
                  module.write(code)
                  module.write('\n')
                     
MODULE_FROM_NOTEBOOK('./ttt_tactics.py',
                     './ttt-tactics.ipynb',
                     'HEAD','GAME','DATA','MODEL','SEARCH')

from ttt_tactics import *

ipynb = 'ttt-tactics-test'

%load_ext tensorboard


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/My Drive/Colab Notebooks
'!!!.ipynb'    ttt-tactics	        ttt_tactics.py
 __pycache__   ttt-tactics.ipynb        ttt-tactics-test.ipynb
 ttt-3.ipynb   ttt-tactics-play.ipynb   ttt-tactics-train.ipynb
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 17888159566824924358
,
 name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 9091487444685002401
physical_device_desc: "device: XLA_CPU device"
]
tensorflow: 2.3.0
DIR = ./ttt-tactics
MODULE =  ttt_tactics
<function savesamples at 0x7f5f8e3c0400>
<function loadsamples at 0x7f5f8e3c0620>
<class 'ttt_tactics.AlphaZeroModel'>
<function AlphaZeroPolicy at 0x7f5f8e3c0d08>


In [14]:
# !pip install git+https://github.com/jackd/run_length_encoding@master

  Cloning https://github.com/jackd/run_length_encoding (to revision master) to /tmp/pip-req-build-trha9ln6
  Running command git clone -q https://github.com/jackd/run_length_encoding /tmp/pip-req-build-trha9ln6
  Created wheel for rle: filename=rle-0.0.1-cp36-none-any.whl size=10439 sha256=1a07f4104dd8b668094c45ea2a0eaad493112988300d40f74eba195113211dd4
  Stored in directory: /tmp/pip-ephem-wheel-cache-ye3z8l0c/wheels/e2/29/30/fe3551581bb31aefa72045e0ad2e99d213703004402201dbb7
Successfully built rle


In [3]:
from rle.np_impl import dense_to_rle

@autograph.experimental.do_not_convert
def loadsamples(files, buffer_size=1024, batch_size=64, seed=None):
    proto = { 'state'  : FixedLenFeature([size_s], int64),
              'probs'  : FixedLenFeature([num_a],  float),
              'reward' : FixedLenFeature([1],      int64) }
    def loadsample(sample):
        sample = io.parse_single_example(sample,proto)
        input  = {'state'  : sample['state']}
        output = {'value'  : sample['reward'], 
                  'policy' : sample['probs']}
        return input,output 
    dataset = data.TFRecordDataset(files)            
    dataset = dataset.map(loadsample, num_parallel_calls=AUTOTUNE)
    return dataset


path    = f'{DIR}/data-1601999616.0.tfrec'       
files   = sortedfiles(path)
dataset = loadsamples(files)
actor   = 0
i       = 0
for input,output in dataset:
    # print(input['state'].shape,output['policy'].shape,output['value'].shape)      
    s = input['state']
    t = turn(s)
    p = player(s)
    k = digits(s)
    if p != actor:
        actor = p
        print(i,'turn',t,'player',p)
        print(s.shape,s)
        print(strings.length(k),k)
    #pprint(output['policy'])
    #pprint(output['value'])
        if i > 100:
            break
        i += 1



0 turn 0 player 1
TensorShape([91]) [0 0 0 ... 0 0 0]
90 "111111111111111111111111111111111111111111111111111111111111111111111111111111111111111111"
1 turn 1 player 2
TensorShape([91]) [-1 -1 -1 ... 0 0 1]
90 "000000000000000000000000000000000000000000000000000020000000000111111111000000000111111111"
2 turn 2 player 1
TensorShape([91]) [-1 -1 -1 ... 0 0 2]
90 "000000000000000000000000000000000020000000000111111111000000000000003000000000000111111111"
3 turn 3 player 2
TensorShape([91]) [-1 -1 -1 ... 0 0 3]
90 "000000000000000000000000000000000020111111111000020000000000000000003000000000000111111111"
4 turn 4 player 1
TensorShape([91]) [-1 -1 -1 ... 0 0 4]
90 "000000000000000000000000000000000020000000003000020000000000000000003000111111111111111111"
5 turn 5 player 2
TensorShape([91]) [-1 -1 -1 ... 0 0 5]
90 "000000000000000000000000000000020000000000300111111121000002000000300000000000000111111111"
6 turn 6 player 1
TensorShape([91]) [-1 -1 -1 ... 0 0 6]
90 "000000000000000000000000

In [ ]:
%tensorboard --logdir "./tensorboard/"

In [ ]:
model = AlphaZeroModel()
models = sortedfiles(f'{DIR}/model-*.h5')
model.load(models[-1])

pi = AlphaZeroPolicy(model, times=100, cp=0, tau=0)
ratios12 = testgames({1:pi,2:randompi}, iters=100)
ratios21 = testgames({2:pi,1:randompi}, iters=100)

NOTIFY(f'{str(model)} tested, {ratios12} vs {ratios21}, shell={str(ipython)}', title=ipynb, priority=1)






./ttt-tactics/model-008.h5 Oct 10 2020 13:18:11
testgames 1:searchpi(model-008,times=100,cp=0,tau=0) vs 2:randompi
  3/100 [..............................] - ETA: 1:28:25 - agent: 0.3333 - rival: 0.3333 - draw: 0.3333    